<a href="https://colab.research.google.com/github/Yiting916/GCP/blob/main/scikit_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

安裝Dataflow、Pubsub、Bigquery等SDK模組

In [1]:
!pip install google-api-core --quiet
!pip install google-cloud-pubsub google-cloud-bigquery-storage --quiet
!pip install apache-beam[gcp,dataframe] --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.5/274.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.1/526.1 kB 18.6 MB/s e

授予Colab筆記本訪問管理GCP權限

In [2]:
from google.colab import auth
auth.authenticate_user()


導入實驗所需模組

In [6]:
import pickle
from sklearn import linear_model
from typing import Tuple

import numpy as np
import apache_beam as beam

from apache_beam.ml.inference.sklearn_inference import ModelFileType
from apache_beam.ml.inference.sklearn_inference import SklearnModelHandlerNumpy
from apache_beam.ml.inference.base import KeyedModelHandler
from apache_beam.ml.inference.base import PredictionResult
from apache_beam.ml.inference.base import RunInference
from apache_beam.options.pipeline_options import PipelineOptions


# 定義常數: 設定專案/值區 ID

In [4]:
import os

# 定義常數（Constants）
project = "tibame-gad245-14-0319"
bucket = "tibame-gad245-14-0319"

# 設定專案 ID。
os.environ['GOOGLE_CLOUD_PROJECT'] = project

# 建立 scikit-learn 模型

建立線性迴歸模型

In [7]:
# 準備訓練 sklearn 模型的輸入資料（5 倍表）。
x = np.arange(0, 100, dtype=np.float32).reshape(-1, 1)
y = (x * 5).reshape(-1, 1)

def train_and_save_model(x, y, model_file_name):
  """訓練線性回歸模型並儲存至檔案。"""
  regression = linear_model.LinearRegression()
  regression.fit(x, y)

  with open(model_file_name, 'wb') as f:
      pickle.dump(regression, f)

# 訓練並儲存 5 倍表模型。
five_times_model_filename = 'sklearn_5x_model.pkl'
train_and_save_model(x, y, five_times_model_filename)

# 訓練並儲存 10 倍表模型。
ten_times_model_filename = 'sklearn_10x_model.pkl'
train_and_save_model(x, y, ten_times_model_filename)
y = (x * 10).reshape(-1, 1)
train_and_save_model(x, y, 'sklearn_10x_model.pkl')



# Dataflow 測試

建立 BigQuey 資料來源

In [11]:
%pip install --upgrade google-cloud-bigquery --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.9/247.9 kB 3.8 MB/s eta 0:00:00


In [14]:
!gcloud config set project $project

Updated property [core/project].


In [18]:
## 將資料填入 BigQuery 資料表

from google.cloud import bigquery

client = bigquery.Client(project=project)

# 確保 dataset_id 在專案中是唯一的。
dataset_id = '{project}.maths'.format(project=project)
dataset = bigquery.Dataset(dataset_id)

# 根據專案設定修改位置。
dataset.location = 'US'
dataset = client.create_dataset(dataset, exists_ok=True)

# BigQuery 資料集中的資料表名稱。
table_name = 'maths_problems_1'

query = """
    CREATE OR REPLACE TABLE
      {project}.maths.{table} ( key STRING OPTIONS(description="A unique key for the maths problem"),
    value FLOAT64 OPTIONS(description="Our maths problem" ) );
    INSERT INTO maths.{table}
    VALUES
      ("first_example", 105.00),
      ("second_example", 108.00),
      ("third_example", 1000.00),
      ("fourth_example", 1013.00)
""".format(project=project, table=table_name)

create_job = client.query(query)
create_job.result()


# 法1: 讀取 BigQuery 資料進行預測

建立 RunInference 管道

In [19]:
sklearn_model_handler = SklearnModelHandlerNumpy(model_uri=five_times_model_filename)

# 設定 Dataflow 管道選項，並指定暫存位置。
pipeline_options = PipelineOptions().from_dictionary(
                                      {'temp_location': f'gs://{bucket}/tmp'})

# 定義 BigQuery 資料表規格。
table_name = 'maths_problems_1'
table_spec = f'{project}:maths.{table_name}'

with beam.Pipeline(options=pipeline_options) as p:
  (
      p
      | "從 BigQuery 讀取資料" >> beam.io.ReadFromBigQuery(table=table_spec)
      | "提取輸入值" >> beam.Map(lambda x: [x['value']])
      | "執行 Sklearn 推論" >> RunInference(model_handler=sklearn_model_handler)
      | beam.Map(print)
  )



PredictionResult(example=[105.0], inference=array([525.]), model_id='sklearn_5x_model.pkl')
PredictionResult(example=[108.0], inference=array([540.]), model_id='sklearn_5x_model.pkl')
PredictionResult(example=[1013.0], inference=array([5065.]), model_id='sklearn_5x_model.pkl')
PredictionResult(example=[1000.0], inference=array([5000.]), model_id='sklearn_5x_model.pkl')


# 法2: 讀取鍵值資料進行預測

In [20]:
sklearn_model_handler = SklearnModelHandlerNumpy(model_uri=five_times_model_filename)
keyed_sklearn_model_handler = KeyedModelHandler(sklearn_model_handler)

# 設定 Dataflow 管道選項，並指定暫存位置。
pipeline_options = PipelineOptions().from_dictionary(
                                      {'temp_location': f'gs://{bucket}/tmp'})

with beam.Pipeline(options=pipeline_options) as p:
  (
  p
  | "從 BigQuery 讀取資料" >> beam.io.ReadFromBigQuery(table=table_spec)
  | "提取輸入值" >> beam.Map(lambda x: (x['key'], [x['value']]))
  | "執行 Sklearn 推論" >> RunInference(model_handler=keyed_sklearn_model_handler)
  | beam.Map(print)
  )



('first_example', PredictionResult(example=[105.0], inference=array([525.]), model_id='sklearn_5x_model.pkl'))
('second_example', PredictionResult(example=[108.0], inference=array([540.]), model_id='sklearn_5x_model.pkl'))
('fourth_example', PredictionResult(example=[1013.0], inference=array([5065.]), model_id='sklearn_5x_model.pkl'))
('third_example', PredictionResult(example=[1000.0], inference=array([5000.]), model_id='sklearn_5x_model.pkl'))


# 法3: 建立多模型管道進行預測

In [21]:
from typing import Tuple

def format_output(run_inference_output) -> str:
  """從 RunInference 的輸出提取 scikit-learn 預測結果。"""
  key, prediction_result = run_inference_output
  example = prediction_result.example[0]
  prediction = prediction_result.inference[0]
  return f"鍵值 = {key}, 範例 = {example} -> 預測結果 {prediction}"

five_times_model_handler = KeyedModelHandler(
    SklearnModelHandlerNumpy(model_uri=five_times_model_filename))
ten_times_model_handler = KeyedModelHandler(
    SklearnModelHandlerNumpy(model_uri=ten_times_model_filename))

# 設定 Dataflow 管道選項，並指定暫存位置。
pipeline_options = PipelineOptions().from_dictionary(
                                      {'temp_location': f'gs://{bucket}/tmp'})

with beam.Pipeline(options=pipeline_options) as p:
  inputs = (p
    | "從 BigQuery 讀取資料" >> beam.io.ReadFromBigQuery(table=table_spec))

  five_times = (inputs
    | "提取 5 倍運算" >> beam.Map(lambda x: ('{} {}'.format(x['key'], '* 5'), [x['value']]))
    | "執行 5 倍推論" >> RunInference(model_handler = five_times_model_handler))

  ten_times = (inputs
    | "提取 10 倍運算" >> beam.Map(lambda x: ('{} {}'.format(x['key'], '* 10'), [x['value']]))
    | "執行 10 倍推論" >> RunInference(model_handler = ten_times_model_handler))

  _ = ((five_times, ten_times)
    | "合併結果" >> beam.Flatten()
    | "格式化輸出" >> beam.Map(format_output)
    | "輸出結果" >> beam.Map(print))



鍵值 = first_example * 5, 範例 = 105.0 -> 預測結果 525.0
鍵值 = second_example * 5, 範例 = 108.0 -> 預測結果 540.0
鍵值 = fourth_example * 5, 範例 = 1013.0 -> 預測結果 5065.0
鍵值 = third_example * 5, 範例 = 1000.0 -> 預測結果 5000.0
鍵值 = first_example * 10, 範例 = 105.0 -> 預測結果 1050.0
鍵值 = second_example * 10, 範例 = 108.0 -> 預測結果 1080.0
鍵值 = fourth_example * 10, 範例 = 1013.0 -> 預測結果 10130.0
鍵值 = third_example * 10, 範例 = 1000.0 -> 預測結果 10000.0
